# 18: Weather

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib

In [2]:
data_month_double = '07'

## load data

In [3]:
vrs_data = pd.read_parquet('/Volumes/T7/Master/Processed Data/'+data_month_double+'/17_VRS_Tier_Next_Merge_Complete_NEU_NEU_TEST.parquet', engine='pyarrow')

In [4]:
# transform geometry
point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["station_point"], crs=4326))
buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["buffer_zone"], crs=4326))

In [5]:
# drop geometry columns in wrong format
vrs_data = vrs_data.drop(columns=["station_point", "buffer_zone"])

In [6]:
# replace geometry columns with correct format
vrs_data['station_point'] = point
vrs_data['buffer_zone'] = buffer

In [7]:
vrs_data

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,tier_trips_count,tier_trips_id,tier_trips_end_at_station_count,tier_trips_end_at_station_id,nextbike_trips_count,nextbike_trips_id,nextbike_trips_end_at_station_count,nextbike_trips_end_at_station_id,station_point,buffer_zone
0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],0,,0,[],POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ..."
1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],0,,0,[],POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ..."
2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],0,,0,[],POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ..."
3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],0,,0,[],POINT (7.11452 50.72175),"POLYGON ((7.11727 50.72208, 7.11731 50.72191, ..."
4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,61,572-16-001-695.1.12:232100-19-2_48C87B00-9EEF-...,Bad Godesberg Stadthalle,1,...,0,,0,[],0,,0,[],POINT (7.11158 50.72556),"POLYGON ((7.11433 50.72589, 7.11437 50.72572, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ..."
873987,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ..."
873988,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ..."
873989,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,151,6040477-604-006-1382.2.21:232600-49-1_6CE5A8D8...,Hersel,0,...,0,,0,[],0,,0,[],POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ..."


In [8]:
# load weather data
weather_data = pd.read_csv('/Volumes/T7/Master/Raw Data/weather_bonn.csv')

## cut and merge with main data

In [9]:
# only keep actual weather data
weather_data = weather_data.filter(like='current')

In [10]:
# drop column
weather_data.drop(columns=["current_snow.1h"], inplace=True)

In [11]:
weather_data

,current_time,current_temp,current_feels_like,current_cloudiness,current_visibility,current_wind_speed,current_description,current_precipitation_volume
0,2024-01-03 07:01:08+01:00,10.07,9.20,75,10000,6.69,broken clouds,NaN
1,2024-01-02 20:01:06+01:00,11.18,10.49,100,10000,6.26,overcast clouds,NaN
2,2024-01-02 17:01:07+01:00,10.39,9.76,75,10000,5.66,light rain,0.93
3,2024-01-03 03:01:08+01:00,10.15,9.23,75,10000,9.26,broken clouds,NaN
4,2024-01-03 19:01:09+01:00,9.74,7.41,75,10000,4.63,broken clouds,NaN
...,...,...,...,...,...,...,...,...
8931,2023-01-02 11:01:36+01:00,11.67,11.14,100,10000,4.12,light intensity shower rain,{'1h': 0.71}
8932,2023-01-02 12:01:37+01:00,11.40,10.89,20,10000,5.14,light rain,{'1h': 0.64}
8933,2023-01-02 13:01:37+01:00,11.27,10.78,40,10000,5.14,light rain,{'1h': 0.14}
8934,2023-01-02 14:01:40+01:00,11.20,10.73,75,10000,2.06,light rain,{'1h': 0.67}


In [12]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8936 entries, 0 to 8935
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   current_time                  8936 non-null   object 
 1   current_temp                  8936 non-null   float64
 2   current_feels_like            8936 non-null   float64
 3   current_cloudiness            8936 non-null   int64  
 4   current_visibility            8936 non-null   int64  
 5   current_wind_speed            8936 non-null   float64
 6   current_description           8936 non-null   object 
 7   current_precipitation_volume  1126 non-null   object 
dtypes: float64(3), int64(2), object(3)
memory usage: 558.6+ KB


In [13]:
# convert into datetime
weather_data['current_time'] = pd.to_datetime(weather_data['current_time'], utc = True)


In [14]:
# delete timezone
weather_data['current_time'] = weather_data['current_time'].dt.tz_localize(None)


In [15]:

# add one hour to get back to original time (Berlin time)
weather_data['current_time'] = weather_data['current_time'] + pd.Timedelta(hours=1)


In [16]:
# only 2023 data
weather_data = weather_data[weather_data['current_time'].dt.year == 2023]


In [17]:
# sort weather data
sorted_weather_data = weather_data.sort_values('current_time')


In [18]:
# reset index
sorted_weather_data = sorted_weather_data.reset_index(drop=True)

In [19]:
# drop Null values in vrs data (needed to merge with weather)
#vrs_data.dropna(subset=['actual_departure_time', 'actual_arrival_time'], inplace=True)

In [20]:
# sort by actual departure time (needed for merge)
vrs_data_sorted = vrs_data.sort_values('scheduled_arrival_time')


In [21]:
# merge on nearest time
merged_data = pd.merge_asof(vrs_data_sorted, sorted_weather_data, left_on='scheduled_arrival_time', right_on='current_time')

In [22]:
merged_data

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,station_point,buffer_zone,current_time,current_temp,current_feels_like,current_cloudiness,current_visibility,current_wind_speed,current_description,current_precipitation_volume
0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11525,1628-16-001-371.1.11:243200-38-257_57C77E14-81...,Severinstr.,0,...,POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",2023-07-01 00:01:08,16.01,15.52,0,10000,3.60,clear sky,NaN
1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11525,1628-16-001-371.1.11:243200-38-257_57C77E14-81...,Severinstr.,0,...,POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",2023-07-01 00:01:08,16.01,15.52,0,10000,3.60,clear sky,NaN
2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11525,1628-16-001-371.1.11:243200-38-257_57C77E14-81...,Severinstr.,0,...,POINT (7.15388 50.68785),"POLYGON ((7.15662 50.68818, 7.15666 50.68801, ...",2023-07-01 00:01:08,16.01,15.52,0,10000,3.60,clear sky,NaN
3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11525,1628-16-001-371.1.11:243200-38-257_57C77E14-81...,Severinstr.,0,...,POINT (7.14823 50.69139),"POLYGON ((7.15098 50.69173, 7.15102 50.69156, ...",2023-07-01 00:01:08,16.01,15.52,0,10000,3.60,clear sky,NaN
4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11525,1628-16-001-371.1.11:243200-38-257_57C77E14-81...,Severinstr.,0,...,POINT (7.14150 50.69677),"POLYGON ((7.14425 50.69711, 7.14429 50.69693, ...",2023-07-01 00:01:08,16.01,15.52,0,10000,3.60,clear sky,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873986,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,27304,661573-66-006-1788.1.11:261200-24-177_9356ED15...,Bonn Hbf,0,...,POINT (7.11744 50.73923),"POLYGON ((7.12019 50.73957, 7.12023 50.73939, ...",2023-08-01 02:01:08,17.20,17.46,75,10000,4.12,light intensity drizzle rain,0.52
873987,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,27314,661575-66-006-687.4.13:263600-3-1_09272580-D40...,Brühler Str.,0,...,POINT (7.06884 50.74026),"POLYGON ((7.07160 50.74060, 7.07163 50.74043, ...",2023-08-01 02:01:08,17.20,17.46,75,10000,4.12,light intensity drizzle rain,0.52
873988,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,27304,661573-66-006-1788.1.11:261200-24-177_9356ED15...,Bonn Hbf,0,...,POINT (7.10248 50.73747),"POLYGON ((7.10523 50.73780, 7.10527 50.73763, ...",2023-08-01 02:01:08,17.20,17.46,75,10000,4.12,light intensity drizzle rain,0.52
873989,66,6,66,0,Tram,SWB Stadtwerke Bonn Verkehrs GmbH,27304,661573-66-006-1788.1.11:261200-24-177_9356ED15...,Bonn Hbf,0,...,POINT (7.09536 50.73639),"POLYGON ((7.09811 50.73673, 7.09815 50.73656, ...",2023-08-01 02:01:08,17.20,17.46,75,10000,4.12,light intensity drizzle rain,0.52


In [23]:
gdf = gpd.GeoDataFrame(merged_data, geometry='station_point')

In [24]:
gdf.to_parquet('/Volumes/T7/Master/Processed Data/'+data_month_double+'/18_Merged_with_Weather_NEU_NEU_TEST.parquet')